Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Automated Machine Learning
_**ディープラーンニングを利用したテキスト分類**_

## Contents
1. [事前準備](#1.-事前準備)
1. [自動機械学習 Automated Machine Learning](2.-自動機械学習-Automated-Machine-Learning)
1. [結果の確認](#3.-結果の確認)

## 1. 事前準備

本デモンストレーションでは、AutoML の深層学習の機能を用いてテキストデータの分類モデルを構築します。  
AutoML には Deep Neural Network が含まれており、テキストデータから **Embedding** を作成することができます。GPU サーバを利用することで **BERT** が利用されます。

深層学習の機能を利用するためには Azure Machine Learning の Enterprise Edition が必要になります。詳細は[こちら](https://docs.microsoft.com/en-us/azure/machine-learning/concept-editions#automated-training-capabilities-automl)をご確認ください。

## 1.1 Python SDK のインポート

Azure Machine Learning の Python SDK などをインポートします。

In [ ]:
import logging
import os
import shutil

import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.run import Run
from azureml.widgets import RunDetails
from azureml.core.model import Model 
from azureml.train.automl import AutoMLConfig
from sklearn.datasets import fetch_20newsgroups

In [ ]:
from azureml.automl.core.featurization import FeaturizationConfig

Azure ML Python SDK のバージョンが 1.8.0 以上になっていることを確認します。

In [ ]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

## 1.2 Azure ML Workspace との接続

In [ ]:
ws = Workspace.from_config()

In [ ]:

# 実験名の指定
experiment_name = 'livedoor-news-classification-BERT'

experiment = Experiment(ws, experiment_name)

output = {}
#output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

## 1.3 計算環境の準備

BERT を利用するための GPU の `Compute Cluster` を準備します。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Compute Cluster の名称
amlcompute_cluster_name = "gpucluster"

# クラスターの存在確認
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    
except ComputeTargetException:
    print('指定された名称のクラスターが見つからないので新規に作成します.')
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6_V3",
                                                           max_nodes = 4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## 1.4 学習データの準備
今回は [livedoor New](https://www.rondhuit.com/download/ldcc-20140209.tar.gz) を学習データとして利用します。ニュースのカテゴリー分類のモデルを構築します。  

In [ ]:
target_column_name = 'label'  # カテゴリーの列 
feature_column_name = 'text'  # ニュース記事の列

In [ ]:
train_dataset = Dataset.get_by_name(ws, "livedoor").keep_columns(["text","label"])
train_dataset.take(5).to_pandas_dataframe()

# 2. 自動機械学習 Automated Machine Learning
## 2.1 設定と制約条件

自動機械学習 Automated Machine Learning の設定と学習を行っていきます。

In [ ]:
from azureml.automl.core.featurization import FeaturizationConfig
featurization_config = FeaturizationConfig()
# テキストデータの言語を指定します。日本語の場合は "jpn" と指定します。
featurization_config = FeaturizationConfig(dataset_language="jpn") # 英語の場合は下記をコメントアウトしてください。

In [ ]:
# 明示的に `text` の列がテキストデータであると指定します。
featurization_config.add_column_purpose('text', 'Text')
#featurization_config.blocked_transformers = ['TfIdf','CountVectorizer']  # BERT のみを利用したい場合はコメントアウトを外します

In [ ]:
# 自動機械学習の設定
automl_settings = {
    "experiment_timeout_hours" : 2,  # 学習時間 (hour)
    "primary_metric": 'accuracy',  # 評価指標
    "max_concurrent_iterations": 4,  # 計算環境の最大並列数 
    "max_cores_per_iteration": -1,
    "enable_dnn": True, # 深層学習を有効
    "enable_early_stopping": False,
    "validation_size": 0.2,
    "verbosity": logging.INFO,
    "force_text_dnn": True,
    #"n_cross_validations": 5,
}

automl_config = AutoMLConfig(task = 'classification', 
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data=train_dataset,
                             label_column_name=target_column_name,
                             featurization=featurization_config,
                             **automl_settings
                            )

## 2.2 モデル学習

自動機械学習 Automated Machine Learning によるモデル学習を開始します。

In [ ]:
automl_run = experiment.submit(automl_config, show_output=False)

In [ ]:
# run_id を出力
automl_run.id

In [ ]:
# Azure Machine Learning studio の URL を出力
automl_run

In [ ]:
# # 途中でセッションが切れた場合の対処
# from azureml.train.automl.run import AutoMLRun
# ws = Workspace.from_config()
# experiment = ws.experiments['livedoor-news-classification-BERT'] 
# run_id = "AutoML_e69a63ae-ef52-4783-9a9f-527d69d7cc9d"
# automl_run = AutoMLRun(experiment, run_id = run_id)
# automl_run


## 2.3 モデルの登録

In [ ]:
# 一番精度が高いモデルを抽出
best_run, fitted_model = automl_run.get_output()

In [ ]:
# モデルファイル(.pkl) のダウンロード
model_dir = '../model'
best_run.download_file('outputs/model.pkl', model_dir + '/model.pkl')

In [ ]:
# Azure ML へモデル登録
model_name = 'livedoor-model'
model = Model.register(model_path = model_dir + '/model.pkl',
                       model_name = model_name,
                       tags=None,
                       workspace=ws)

# 3. テストデータに対する予測値の出力

In [ ]:
from sklearn.externals import joblib
trained_model = joblib.load(model_dir + '/model.pkl')

In [ ]:
trained_model

In [ ]:
test_dataset = Dataset.get_by_name(ws, "livedoor").keep_columns(["text"])
predicted = trained_model.predict_proba(test_dataset.to_pandas_dataframe())

# 4. モデルの解釈
一番精度が良かったチャンピョンモデルを選択し、モデルの解釈をしていきます。  
モデルに含まれるライブラリを予め Python 環境にインストールする必要があります。[automl_env.yml](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/automl_env.yml)を用いて、conda の仮想環境に必要なパッケージをインストールしてください。

In [ ]:
# 特徴量エンジニアリング後の変数名の確認
fitted_model.named_steps['datatransformer'].get_json_strs_for_engineered_feature_names()
#fitted_model.named_steps['datatransformer']. get_engineered_feature_names ()

In [ ]:
# 特徴エンジニアリングのプロセスの可視化
text_transformations_used = []
for column_group in fitted_model.named_steps['datatransformer'].get_featurization_summary():
    text_transformations_used.extend(column_group['Transformations'])
text_transformations_used